# imports

In [ ]:
import pandas as pd
import numpy as np
!pip install transformers==4.28.0
!pip install datasets
from datasets import load_dataset
from transformers import BertTokenizer, BertForMaskedLM
import torch
from transformers import AutoTokenizer


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.1 MB/s eta 0:00:0

In [ ]:
print('a')

a




1.   ליצור דתא סט של csv 
שבתוחו יש את המידע שלי
DONE
2.   להמיר את המידע ל tourch dataset
3. לראות אני דוחף את זה למודל והתוצאות יוצאות הגיוניות
4. לדחוף אם דתא לואדר ולראות שזה יוצא הגיוני
5. אחרי כל זה לנסות לאמן מודל קטן ולהגיע לאובר פיט, להכין הכל ולראות שהוא נשמר והכל
6. לאחר מכאן לעשות אימון מלא - לאפטם לאימון את הצורההה 
7. להוסיך gradient aculimilation
8. להוסיף amp
9. להוסיף multi gpu
10. בכלל לאמן על gpu 



# importing alephbertgimel

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM


import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer

model = AutoModelForMaskedLM.from_pretrained("imvladikon/alephbertgimmel-small-128")
tokenizer = AutoTokenizer.from_pretrained("imvladikon/alephbertgimmel-small-128")

# creating a data set

In [ ]:
dataset = load_dataset("MetroCat/HEBREW-MIL-CLEAN")['train']

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
my_df = pd.DataFrame(dataset).loc[pd.DataFrame(dataset)['context'].map(lambda x : len(x)>0)].iloc[2:]

In [ ]:
del dataset

In [ ]:
def preprocess_function(data):
    return tokenizer(
        data,
        truncation=True, 
        return_overflowing_tokens=True, 
        padding='max_length',
        max_length=128,
        stride=0
)

In [ ]:
#my_df['comb and tokenize'] = 
my_df['comb and tokenize'] = my_df['context'].map(preprocess_function)

In [ ]:
my_df['input_ids'] = my_df['comb and tokenize'].map(lambda x: x['input_ids'])
my_df['token_type_ids'] = my_df['comb and tokenize'].map(lambda x: x['token_type_ids'])
my_df['attention_mask'] = my_df['comb and tokenize'].map(lambda x: x['attention_mask'])

In [ ]:
my_df = my_df.drop(columns=['context'])

In [ ]:
my_df = my_df.drop(columns=['comb and tokenize'])

In [ ]:
file_name_arr =  []
input_id_arr = []
token_type_ids_arr = []
attention_mask_arr  =[]
from tqdm import tqdm
for i in tqdm(range(0,len(my_df))):
  for j in range(len(my_df.iloc[i]['input_ids'])):
    file_name_arr.append(my_df.iloc[i]['file_name'])
    input_id_arr.append(my_df.iloc[i]['input_ids'][j])
    token_type_ids_arr.append(my_df.iloc[i]['token_type_ids'][j])
    attention_mask_arr.append(my_df.iloc[i]['attention_mask'][j])

100%|██████████| 1073/1073 [00:55<00:00, 19.18it/s]


In [ ]:
del my_df

In [ ]:

data  =pd.DataFrame({'input_id': input_id_arr,'token_type_ids': token_type_ids_arr,'attention_mask':attention_mask_arr})

In [ ]:
data  =data.sample(n=round(len(data)/40), random_state=1)

In [ ]:
data.to_csv('eval.csv')

In [ ]:
import pandas as pd
import numpy as np
!pip install transformers==4.28.0
!pip install datasets
from datasets import load_dataset
from transformers import BertTokenizer, BertForMaskedLM
import torch
from transformers import AutoTokenizer


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 113.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.8 MB/s eta 0:00:

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM


import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer

model = AutoModelForMaskedLM.from_pretrained("imvladikon/alephbertgimmel-small-128")
tokenizer = AutoTokenizer.from_pretrained("imvladikon/alephbertgimmel-small-128")

In [ ]:
data = pd.read_csv('eval.csv')

In [ ]:
data =data.rename(columns ={'input_id': 'input_ids'})

In [ ]:
data = data[3000:]

In [ ]:
import ast
data['input_ids'] = data['input_ids'].map(lambda x : ast.literal_eval(re.sub(r'(\d+)\s', r'\1, ', x)))
data['token_type_ids'] = data['token_type_ids'].astype(str).map(lambda x : ast.literal_eval(re.sub(r'(\d+)\s', r'\1, ', x)))
data['attention_mask'] = data['attention_mask'].map(lambda x :ast.literal_eval(re.sub(r'(\d+)\s', r'\1, ', x)))


In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(data)

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15, return_tensors="pt")

In [ ]:
from transformers import TrainingArguments,Trainer
training_args = TrainingArguments(

    output_dir="test_great_model",

    evaluation_strategy="steps",
    eval_steps =5,



    push_to_hub=False,
auto_find_batch_size =  True,
    per_device_eval_batch_size =64,

#   fp16  = True
# warmup_steps =0.1
)

trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=dataset,

    eval_dataset=dataset,

    data_collator=data_collator,
    

)


In [ ]:
import math

eval_results = trainer.evaluate()

print(f"crossentropy: {eval_results['eval_loss']:.2f}")

crossentropy: 88.16


In [ ]:
print(f"crossentropy: {eval_results['eval_loss']:.2f}")

crossentropy: 4.48


realy high

# increaseing the embedding matrix size 

In [ ]:
## we will use full pre training becouse our model is good or rubust enougth for 

# fine tuning to see overfit in torch lightning

# evaluating on text